In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 4.3 MB/s 


In [3]:
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import scipy.io as sio
import pandas as pd
import time
import cv2
from skimage.transform import resize
from IPython.display import clear_output

In [4]:
def one_hot(vec):
  items=np.sort(pd.unique(vec))
  n_class=np.shape(items)[0]
  zeros=np.zeros((vec.size, n_class))
  for n,i in enumerate(items):
    rows=np.where(vec==i)[0]
    zeros[rows,n]=1
  return zeros.astype('float32'), n_class

In [5]:
def get_index(vec_lab, seed=1996, split_train=0.6, validation=False):
  train=[]
  valid=[]
  test =[]

  spv=(1-split_train)/2.
  for i in range(2):
    ind_clss=np.where(vec_lab[:,i]==1)[0]
    sz=len(ind_clss)
    np.random.seed(seed)
    ramd=np.random.choice(sz, sz, replace=False)
    train=np.append(train, ind_clss[ramd[:int(sz*split_train)]])
    valid=np.append(valid, ind_clss[ramd[int(sz*split_train):int(sz*(split_train+spv))]])
    test =np.append(test, ind_clss[ramd[int(sz*(split_train+spv)):]])
  train=train[np.random.choice(len(train), len(train), replace=False)].astype('int')
  valid=valid[np.random.choice(len(valid), len(valid), replace=False)].astype('int')
  test= test[np.random.choice(len(test), len(test), replace=False)].astype('int')
  if validation:
    return train, valid, test
  else:
    return np.concatenate((train,valid)), test

In [6]:
import tensorflow_addons as tfa
def augmentation_f(data, labx):
  images=tf.concat([data,
                  tfa.image.rotate(data, angles=np.pi/2),
                  tfa.image.rotate(data, angles=np.pi),
                  tfa.image.rotate(data, angles=np.pi*3/2)], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [7]:
def get_data_training(indx, ima_tens, labels_v, k_fold='none', number_folds=10, augm=False):
  if k_fold!='none':
    sz=len(indx)/number_folds
    valid=indx[int(k_fold*sz):int((k_fold+1)*sz)]
    train=np.concatenate((indx[:int(k_fold*sz)],indx[int((k_fold+1)*sz):])) 

    x_data=tf.image.grayscale_to_rgb(tf.convert_to_tensor(np.array(ima_tens)[train]))
    v_data=tf.image.grayscale_to_rgb(tf.convert_to_tensor(np.array(ima_tens)[valid]))
    y_data=tf.convert_to_tensor(np.array(labels_v)[train])
    vy_dat=tf.convert_to_tensor(np.array(labels_v)[valid])

    if augm:
      x_data, y_data=augmentation_f(x_data, y_data)
      v_data, vy_dat=augmentation_f(v_data, vy_dat)

    return x_data, v_data, y_data, vy_dat
  else:
    x_data=tf.image.grayscale_to_rgb(tf.convert_to_tensor(np.array(ima_tens)[indx]))
    y_data=tf.convert_to_tensor(np.array(labels_v)[indx])
    if augm:
      x_data, y_data=augmentation_f(x_data, y_data)
    return x_data, y_data

In [8]:
def get_data(sequences='T1'): #'FLAIR','T1_Gd'
  data_dir=('/content/drive/MyDrive/Brain_tumors_v2/Datasets/TCIA_LGG/x_x.mat')
  mats=sio.loadmat(data_dir.replace('x_x', sequences))
  images=mats['images']
  images=images.reshape((np.shape(images)[0],np.shape(images)[1],np.shape(images)[2],1))
  labels=(mats['size_tumor'][0]!=0).astype('int')
  images=tf.image.resize(images, [240,240], method='nearest')
  labels, n_class=one_hot(labels)
  return images, labels, n_class

In [9]:
from tensorflow.keras import applications as ap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [10]:
def get_model(network, opt='adadelta', loss_name='categorical_crossentropy', input_shape=(240,240,3), classes=2, weights='imagenet'):
  try:
    del model
  except:
    print('done')
  model=Sequential()
  if network=='ResNet50V2':
    model.add(ap.ResNet50V2(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) # The input must have 3 channels
  if network=='EfficientNetB7':
    model.add(ap.EfficientNetB7(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='InceptionResNetV2':
    model.add(ap.InceptionResNetV2(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='InceptionV3':
    model.add(ap.InceptionV3(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='NASNetLarge':
    model.add(ap.NASNetLarge(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes))
  if network=='VGG19':
    model.add(ap.VGG19(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='Xception':
    model.add(ap.Xception(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes))
  if network=='DenseNet121':
    model.add(ap.DenseNet121(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes))
  
  model.add(Dense(classes, activation='softmax'))
  model.compile(optimizer=opt, loss=loss_name, metrics=['acc', tf.keras.metrics.Recall(), tf.keras.metrics.FalsePositives()])
  return model

In [11]:
df = pd.DataFrame(columns=('run_n', 'k_fold', 'network', 'optimizer', 'loss', 'epochs', 'total_parameters', 'time', 'transfer', 'augm', 'Class', 'TP', 'TN', 'FP', 'FN','result_mat'))
df.head()

,run_n,k_fold,network,optimizer,loss,epochs,total_parameters,time,transfer,augm,Class,TP,TN,FP,FN,result_mat


In [12]:
TP=tf.keras.metrics.TruePositives()
TN=tf.keras.metrics.TrueNegatives()
FP=tf.keras.metrics.FalsePositives()
FN=tf.keras.metrics.FalseNegatives()

In [18]:
def run_experiment(images, labels, number_class, augm_con, name='', star_run=0, end_run=30, batch_size=4): 
  for net in Networks:
    name_0='_'.join(['clasification_t', name, str(transferlearning), augm_con, net, optimizer])
    if not os.path.exists(path2+name_0+'.csv'):
      df.to_csv(path2+name_0+'.csv')

    train_id, test_id=get_index(labels)
    x_test, y_test=get_data_training(test_id, images, labels)
    y_test=y_test>=0.5

    for i in range(star_run, end_run): #Numero corridas 
      for j in [loss]: #Funciones de perdida
        cntn=True
        name_m='_'.join([name_0,j,'run',str(i)])  
        print(name_m)

        #Obtener imagenes nuevamente
        x_train, x_valid, y_train, y_valid=get_data_training(train_id, images, labels, k_fold=i%10, augm=(augm_con!='None'))      
        model=get_model(net, classes=number_class, input_shape=list(np.shape(x_train)[1:]),  weights=transferlearning)
        
        try:
          tic = time.time()
          results = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=batch_size, epochs=epochs)
          toc=time.time()-tic
          model.save_weights(pathW+name_m+"w.h5")
        except:
          print('Training error')
          cntn=False

        if cntn:
          sio.savemat(pathW+name_m+'_r.mat', results.history)

          #Validation
          y_hat=np.array(model.predict(x_test))
          sio.savemat(pathW+name_m+'_los_8.mat',{'y_hat': y_hat, 'y_test': np.array(y_test)})
          y_hat=y_hat>=0.5

          for class_i in range(number_class):
            TP.reset_state()
            TN.reset_state()
            FP.reset_state()
            FN.reset_state()        

            TP.update_state(y_test[:,class_i], y_hat[:,class_i])
            TN.update_state(y_test[:,class_i], y_hat[:,class_i])
            FP.update_state(y_test[:,class_i], y_hat[:,class_i])
            FN.update_state(y_test[:,class_i], y_hat[:,class_i])
            total_p=model.count_params()

            #data frame
            df2=pd.read_csv(path2+name_0+'.csv')
            df2=df2.append({'run_n': i,
                            'k_fold': i%10,
                            'network': net,
                            'optimizer': optimizer,
                            'loss': 'categorical_crossentropy',
                            'epochs': epochs,
                            'total_parameters': total_p,
                            'time': toc,
                            'transfer': transferlearning,
                            'augm': augm_con,
                            'Class': class_i,
                            'TP': float(TP.result()),
                            'TN': float(TN.result()),
                            'FP': float(FP.result()),
                            'FN': float(FN.result()),
                            'result_mat': name_m+'_r.mat'} , ignore_index=True)
            df2=df2.drop(df2.columns[:np.where(df2.columns=='run_n')[0][0]], axis=1)
            df2.to_csv(path2+name_0+'.csv')
          del x_train, y_train, x_valid, y_valid, model
          clear_output(wait=True)
    star_run=0

# Run experiment

In [ ]:
Networks=['ResNet50V2', 'EfficientNetB7', 'InceptionResNetV2', 'InceptionV3', 'VGG19', 'Xception', 'DenseNet121']
path2='/content/drive/MyDrive/Brain_tumors_v2/Results/TCIADetection/transfer/results_csv/'
pathW='/content/drive/MyDrive/Brain_tumors_v2/Results/TCIADetection/transfer/Weights/'
augmentation='None'
transferlearning='imagenet'
optimizer='adadelta'
epochs=50
loss='categorical_crossentropy'

#----------------------------------RUN------------------------------------------
seq='T1' #'FLAIR','T1_Gd'
imas, labs, no_cls=get_data(seq)
run_experiment(imas, labs, no_cls, augmentation, seq, star_run=0)